In [1]:
import pandas as pd
import glob,os,pickle
from tqdm import tqdm 

In [7]:
# Получаем ИД пользователя из имени файла
def get_user_id(filename):
    start_pos=(filename.find('\\user'))+5
    end_pos=(filename.find('.csv'))
    return int(filename[start_pos:end_pos])

# Преобразование логов пользователя в таблицу сессий
def user_log_to_session(user_log_df,user_id,session_length,window_size):
    res=pd.DataFrame()
    # Добавляем session_length новых колонок виде site_i, где новая значение колонки берется как колонка site со сдвигом на i строк
    # получаем таблицу вида:
    # site1 time1 site2 time2 site3 time3
    # site2 time2 site3 time3 site4 time4
    for col in range(1,session_length+1):
        shift_index=col-1
        site_col_name='site'+str(col)
        res[site_col_name]=user_log_df.site[shift_index:]
        res[site_col_name]=res[site_col_name].shift(-shift_index)
        timestamp_col_name='time'+str(col)
        res[timestamp_col_name]=user_log_df.timestamp[shift_index:]
        res[timestamp_col_name]=res[timestamp_col_name].shift(-shift_index)
    # Удаляем лишние строки, оставляя только строки которые соответствуюи ширине окна window_size
    res['user_id']=user_id
    ind_tosave=[x for x in range(0,len(user_log_df),window_size)]
    return res.iloc[ind_tosave,:]

# Фуникция читает логи пользователей, склеивает их в единый DataFrame и заменяет названия сайтов на идентификаторы по словарю
def read_and_replace(path_to_csv_files):
    print('1. Читаем исходные логи пользователей')
    all_files = glob.glob(os.path.join(path_to_csv_files, "*.csv"))
    user_log=[]
    # Последовательно читаем все файлы логов пользоваителей в каталоге и склеиваем их в один DataFrame 
    for filename in all_files:
        df_user=pd.read_csv(filename)
        df_user['user_id']=get_user_id(filename)
        user_log.append(df_user)
    all_user_log = pd.concat(user_log, ignore_index=True)
    # Заменяем название сайта на ИД по словарю
    print('2. Заменяем название сайта на ИД по словарю')
    with open('site_dic.pkl', 'rb') as file_pkl:
        site_dic=pickle.load(file_pkl)
    all_user_log.site=all_user_log.site.map(site_dic) 
    return all_user_log
    
# Функция преобразовывает логи пользователя по посещению сайтов в таблицу сессий
def prepare_train_set(all_user_log, session_length,window_size):
    print('3. Формируем сессии')
    sessions=pd.DataFrame()
    for user_id in tqdm(all_user_log.user_id.value_counts().index):
        # Получаем логи одного пользователя
        user_df=all_user_log[all_user_log.user_id==user_id]
        # Формируем для одного пользователя таблицу сессий
        user_session=user_log_to_session(user_df,user_id,session_length,window_size)
        # Добавляем таблицу сессий пользователя в общую таблицу
        sessions=pd.concat([sessions,user_session])
    return sessions

In [11]:
%%time
folder='train/'
# Фуникция формирует таблицу сессий с заданным ширином окна и сохраняет данные в файл
def resize_train(path_to_csv_files, session_length,window_size):
    sessions=prepare_train_set(all_user_log, session_length,window_size)
    with open('train_'+str(window_size)+'_session.pkl', 'wb') as pkl_file:
        pickle.dump(sessions, pkl_file)
all_user_log=read_and_replace(folder)
for window_size in [3,5,7,10]:
    print ('window_size=',window_size)
    resize_train(all_user_log,10,window_size)

1. Читаем исходные логи пользователей
2. Заменяем название сайта на ИД по словарю
window_size= 3
3. Формируем сессии



100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [06:35<00:00,  1.57s/it]


window_size= 5
3. Формируем сессии


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [05:14<00:00,  1.13s/it]


window_size= 7
3. Формируем сессии


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [03:51<00:00,  1.42it/s]


window_size= 10
3. Формируем сессии


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [03:01<00:00,  1.80it/s]


Wall time: 18min 58s


# Прежняя реализация

In [78]:
sessions=pd.DataFrame()
session_length=10
window_size=3
for user_id in tqdm(a.user_id.value_counts().index):
    user_df=a[a.user_id==user_id]
    user_session=user_log_to_session(user_df,user_id,session_length,window_size)
    sessions=pd.concat([sessions,user_session])

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 11.41it/s]


In [16]:
window_size=3
session_length=10
max_sesssion_len=200
# Обрабатываем лог юзера и возвращем индексы записей лога, которые должны войти в одну сессию 
def process_user(user_df,window_size,session_length,max_sesssion_len):
    session_coords=[]
    start_index=user_df.index.min()
    max_index=user_df.index.max()
    while start_index<max_index:
        for pad in range(session_length):
            end_index=start_index+session_length-pad-1
            if end_index>max_index:
                end_index=max_index
            time1=datetime.strptime(user_df.loc[start_index,'timestamp'], '%Y-%m-%d %H:%M:%S')
            time2=datetime.strptime(user_df.loc[end_index,'timestamp'], '%Y-%m-%d %H:%M:%S')
            session_len=(time2-time1).total_seconds()
            if session_len<=max_sesssion_len:
                break
        session_coords.append([start_index,end_index])
        start_index+=window_size
    return (session_coords)

In [24]:
session_coords=[]
for user_id in tqdm(a.user_id.value_counts().index):
    user_df=a[a.user_id==user_id]
    session_coords=process_user(user_df,window_size,session_length,max_sesssion_len)
    sessiond_df=log_to_session(session_coords,user_df,user_id)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.19s/it]


In [21]:
def log_to_session(session_coords,user_df,user_id):
    sessions_num=len(session_coords)
    sessions=np.array([])
    for coord in session_coords:
        session_log=user_df.iloc[coord[0]:coord[1]+1,:][['timestamp','site_id']].as_matrix()
        if len(session_log) < windows_size:
            ar_pad=np.empty(2*(windows_size-len(session_log)))
            ar_pad.fill(np.nan)
            session_log=np.append(session_log, ar_pad)
        sessions=np.append(sessions, session_log)
        sessions=np.append(sessions, user_id)
    col_name=np.array([['time'+str(x),'site'+str(x)] for x in range(1,windows_size+1)]).reshape([1,2*windows_size])[0]
    col_name=np.append(col_name, ['user_id'])
    sessions=pd.DataFrame(sessions.reshape([sessions_num,1+2*windows_size]),columns=col_name)
    return sessions
    
t=log_to_session(session_coords,a,1)

In [194]:
# Взвращаем ИД сайта по его названию
def get_site_id(site,site_dict):
    print ('site:',site)
    return site_dict[site][0]
# Заменяем названия сайтов на ИД, формируем словарь частот сайтов
def get_site_dict(all_user_log):
    # Расчитываем частоту freq для каждого сайта 
    all_user_log['freq'] = all_user_log.groupby('site')['site'].transform('count')
    # Формируем словарь
    site_dict=all_user_log[['site','freq']].drop_duplicates().sort_values(by='freq',ascending=False).reset_index(drop=True)
    site_dict['site_id']=site_dict.index+1
    site_dict.set_index(site_dict.site,inplace=True)
    site_dict['site_freq']=site_dict.apply(lambda row: (row['site_id'],row['freq']), axis=1)
    all_user_log=all_user_log.join(site_dict[['site','site_id']],on=['site'],rsuffix='r')
    site_dict=site_dict.site_freq.to_dict()
    return all_user_log[['timestamp','site_id','user_id']],site_dict


# Функция возвращает подготовленные данные и словарь сайтов
def prepare_train_set2(path_to_csv_files, session_length,window_size):
    # Последовательно читаем все файлы сессий в каталоге и склеиваем их в один DataFrame = sessions_df
    all_files = glob.glob(os.path.join(path_to_csv_files, "*.csv"))
    print('1. Читаем исходные данные')
    user_log=[]
    for filename in all_files:
        df_user=pd.read_csv(filename)
        df_user['user_id']=get_user_id(filename)
        user_log.append(df_user)
    all_user_log = pd.concat(user_log, ignore_index=True)
    print('2. Формируем словарь')
    # Формируем словарь сайтов
    all_user_log,site_dict=get_site_dict(all_user_log)
    print('3. Формируем сессии')
    sessions=pd.DataFrame()
    for user_id in tqdm(all_user_log.user_id.value_counts().index):
        user_df=all_user_log[all_user_log.user_id==user_id]
        user_session=user_log_to_session(user_df,user_id,session_length,window_size)
        sessions=pd.concat([sessions,user_session])
    return sessions,site_dict